In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [4]:
# эмоции
emotions = ['anger', 'disgust',	'fear',	'joy',	'sadness',	'surprise',	'neutral']

In [5]:
# Загрузка данных из Excel файла
input_file = 'C:/Users/vasiliy.vorobev/Documents/Разное/Обучение/Проект/Лог регрессия/valid.csv'
df = pd.read_csv(input_file, usecols=['text', 'anger', 'disgust',	'fear',	'joy',	'sadness',	'surprise',	'neutral'])
df

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,Это в Новом Орлеане?? Я действительно чувствую...,0,0,0,0,0,0,1
1,"Ты знаешь ответ, чувак, ты запрограммирован пе...",0,0,0,1,0,0,1
2,Мне никогда в жизни не было так грустно!,0,0,0,0,1,0,0
3,Экономика находится под жестким контролем и су...,0,0,0,1,0,0,1
4,Он мог бы легко взять настоящую камеру из зако...,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
5421,"Это довольно опасно, когда государство решает,...",0,0,1,0,0,0,0
5422,"Сегодня утром я подала на развод. Надеюсь, он ...",0,0,0,1,0,0,0
5423,"В последний раз, когда это случилось, я просто...",1,0,0,0,0,0,0
5424,"Терпеть не могу этого высокомерного придурка, ...",1,0,0,0,0,0,0


In [6]:
# Удаление Nan
#df = df.dropna()
df[df['text'].isnull()]

,text,anger,disgust,fear,joy,sadness,surprise,neutral


In [7]:
# 3. Провести предобработку поля text для дальнейшего обучения ML модели
# Убедитесь, что вы скачали необходимые наборы данных для NLTK:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Привести к нижнему регистру
    text = text.lower()
    # Удалить лишние символы
    text = re.sub("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+",' ', text)
    # Удалить стоп слова и лемматизировать текст
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['processed_text'] = df['text'].apply(preprocess_text)
df

<>:14: SyntaxWarning: invalid escape sequence '\]'
<>:14: SyntaxWarning: invalid escape sequence '\]'
C:\Users\vasiliy.vorobev\AppData\Local\Temp\ipykernel_17204\3470394433.py:14: SyntaxWarning: invalid escape sequence '\]'
  text = re.sub("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+",' ', text)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vasiliy.vorobev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vasiliy.vorobev\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,anger,disgust,fear,joy,sadness,surprise,neutral,processed_text
0,Это в Новом Орлеане?? Я действительно чувствую...,0,0,0,0,0,0,1,это новом орлеане действительно чувствую это н...
1,"Ты знаешь ответ, чувак, ты запрограммирован пе...",0,0,0,1,0,0,1,знаешь ответ чувак запрограммирован перехватыв...
2,Мне никогда в жизни не было так грустно!,0,0,0,0,1,0,0,жизни грустно
3,Экономика находится под жестким контролем и су...,0,0,0,1,0,0,1,экономика находится жестким контролем субсидир...
4,Он мог бы легко взять настоящую камеру из зако...,0,0,0,1,0,0,0,мог легко взять настоящую камеру законного ист...
...,...,...,...,...,...,...,...,...,...
5421,"Это довольно опасно, когда государство решает,...",0,0,1,0,0,0,0,это довольно опасно государство решает какое в...
5422,"Сегодня утром я подала на развод. Надеюсь, он ...",0,0,0,1,0,0,0,сегодня утром подала развод надеюсь уедет след...
5423,"В последний раз, когда это случилось, я просто...",1,0,0,0,0,0,0,последний это случилось просто сказал «нет» за...
5424,"Терпеть не могу этого высокомерного придурка, ...",1,0,0,0,0,0,0,терпеть могу высокомерного придурка форда онта...


In [8]:
import joblib
from sklearn.metrics import f1_score

#Загрузка векторизатора
loaded_vectorizer = joblib.load('C:/Users/vasiliy.vorobev/Documents/Разное/Обучение/Проект/Лог регрессия/vectorizer.joblib')

# Применение векторизатора к новым данным
X = loaded_vectorizer.transform(df['processed_text'])

# Применение моделей

for emotion in emotions:
    y_val = df[emotion]   
    # Загрузка модели
    model = joblib.load('C:/Users/vasiliy.vorobev/Documents/Разное/Обучение/Проект/Лог регрессия/model' + emotion + '.joblib')
    
    y_pred = model.predict(X)
    y_pred_P = pd.DataFrame(model.predict_proba(X))
    
    #print(y_pred_P[1])
    
    df[emotion+'_pred'] = y_pred
    df[emotion+'_P'] = y_pred_P[1]
    
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    report = classification_report(y_val, y_pred)

    print(emotion)
    print(f"accuracy: {accuracy}")
    print(f"f1 score: {f1}")
    print("Отчет по классификации:")
    print(report)


anger
accuracy: 0.8761518614080354
f1 score: 0.15789473684210525
Отчет по классификации:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93      4709
           1       0.78      0.09      0.16       717

    accuracy                           0.88      5426
   macro avg       0.83      0.54      0.55      5426
weighted avg       0.86      0.88      0.83      5426

disgust
accuracy: 0.9817545152967195
f1 score: 0.019801980198019802
Отчет по классификации:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5329
           1       0.25      0.01      0.02        97

    accuracy                           0.98      5426
   macro avg       0.62      0.50      0.51      5426
weighted avg       0.97      0.98      0.97      5426

fear
accuracy: 0.9826760044231478
f1 score: 0.2033898305084746
Отчет по классификации:
              precision    recall  f1-score   support

           0       0.98

In [25]:
new_order = ['text',	'processed_text', 'anger',	'disgust',	'fear',	'joy',	'sadness',	'surprise',	'neutral', 'anger_pred', 'disgust_pred', 'fear_pred', 'joy_pred', 'sadness_pred', 'surprise_pred', 'neutral_pred', 'anger_P',	'disgust_P', 'fear_P', 'joy_P',	'sadness_P', 'surprise_P', 	'neutral_P']

df = df[new_order]

df.to_excel('C:/Users/vasiliy.vorobev/Documents/Разное/Обучение/Проект/Лог регрессия/valid.xlsx', sheet_name='Данные', index=False, header=True)